**SMA ASSIGNMENT DUTCH_SOCIAL_MEDIA**

In [20]:
!apt-get install git

!git clone https://github.com/sharonmary15/SMA_Dutch_Social_heatmap

%cd SMA_Dutch_Social_heatmap

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git is already the newest version (1:2.34.1-1ubuntu1.15).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
Cloning into 'SMA_Dutch_Social_heatmap'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 4 (delta 0), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (4/4), done.
/content/SMA_Dutch_Social_heatmap/SMA_Dutch_Social_heatmap


In [21]:
from google.colab import drive
drive.mount('/content/drive')

zip_path = "/content/drive/MyDrive/archive.zip"
import zipfile

extract_dir = "/content/extracted_jsons"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_dir)

print("ZIP file extracted to:", extract_dir)

import json
import glob
import pandas as pd

json_files = glob.glob(extract_dir + "/*.json")
print(f"Files Found {len(json_files)} JSON files.")

all_data = []

for file in json_files:
    with open(file, 'r', encoding='utf-8') as f:
        try:
            data = json.load(f)
            if isinstance(data, list):
                all_data.extend(data)
            else:
                all_data.append(data)
        except json.JSONDecodeError:
            print(f"Skipped invalid JSON: {file}")

df = pd.json_normalize(all_data)
print("Data loaded into DataFrame!")
print("Columns available:", df.columns.tolist())
df.head()


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
ZIP file extracted to: /content/extracted_jsons
Files Found 10 JSON files.
Data loaded into DataFrame!
Columns available: ['full_text', 'text_translation', 'created_at', 'screen_name', 'description', 'desc_translation', 'weekofyear', 'weekday', 'day', 'month', 'year', 'location', 'point_info', 'point', 'latitude', 'longitude', 'altitude', 'province', 'hisco_standard', 'hisco_code', 'industry', 'sentiment_pattern', 'subjective_pattern']


,full_text,text_translation,created_at,screen_name,description,desc_translation,weekofyear,weekday,day,month,...,point,latitude,longitude,altitude,province,hisco_standard,hisco_code,industry,sentiment_pattern,subjective_pattern
0,"RT @chrisilluminati: Much like the pandemic, t...","RT @chrisilluminati: Much like the pandemic, t...",1593530905000,hejost67,"Sports, nature, climate, food...","Sports, nature, climate, food ...",27.0,1.0,30.0,6.0,...,"(52.5001698, 5.7480821, 0.0)",52.500170,5.748082,0.0,Flevoland,min,7410,True,0.0000,0.000
1,RT @Dirk_Lauwers: Visie van collega @KBoussauw...,RT @Dirk_Lauwers: Vision @KBoussauw colleagues...,1593672474000,KarelBrits,"If people are not laughing at your goals, your...","If people are not laughing at your goals, your...",27.0,3.0,2.0,7.0,...,"(51.0312293, 4.098112, 0.0)",51.031229,4.098112,0.0,False,None,None,False,0.3000,0.500
2,夏は、甘いのが人気！\n #stay home\nブ\n ラ\n ッ\n ク...,"Summer, sweet popular!\n#stay home\nBreakfast\...",1593672475000,BlackDe74164200,Black Devil 👿再び降臨‼️ 「BLACK DEVIL（ブラックデビル）」は、🇳🇱...,"Black Devil 👿 again descended! ️ ""BLACK DEVIL ...",27.0,3.0,2.0,7.0,...,"(52.5001698, 5.7480821, 0.0)",52.500170,5.748082,0.0,Flevoland,None,None,False,0.0000,0.000
3,"Corona, het mooiste excuus om ons elke nog tas...","Corona, the best excuse to take us any more ta...",1593530908000,Chasidiyah,Je weet nooit wat je voor n ander betekent. Li...,You never know what you mean to one another. I...,27.0,1.0,30.0,6.0,...,None,NaN,NaN,NaN,False,dienst,55220,True,0.0625,0.275
4,@mevrouwadvocaat De Chinesetomatensoep wordt d...,@mevrouwadvocaat Chinese Tomato caught twice. ...,1581174053000,Glijn1,Zoon van de ouwe Glijn en trotse vader van Gli...,Son of the old Glijn and proud father of Glijn...,6.0,5.0,8.0,2.0,...,None,NaN,NaN,0.0,False,advocaat,12110,True,0.0000,0.000


In [22]:
df = df.dropna(subset=["latitude", "longitude"])

df = df[
    (df["latitude"].between(-90, 90)) &
    (df["longitude"].between(-180, 180))
]

print(f"Cleaned records: {len(df)}")


Cleaned records: 134445


In [23]:
import folium
from folium.plugins import HeatMap

map_center = [df["latitude"].mean(), df["longitude"].mean()]
heatmap = folium.Map(location=map_center, zoom_start=6)

heat_data = df[["latitude", "longitude"]].values.tolist()

HeatMap(heat_data, radius=10, blur=15, max_zoom=6).add_to(heatmap)

heatmap.save("geo_heatmap_full.html")
print("Heatmap saved as geo_heatmap_full.html")

Heatmap saved as geo_heatmap_full.html


In [24]:
print(df.columns.tolist())


['full_text', 'text_translation', 'created_at', 'screen_name', 'description', 'desc_translation', 'weekofyear', 'weekday', 'day', 'month', 'year', 'location', 'point_info', 'point', 'latitude', 'longitude', 'altitude', 'province', 'hisco_standard', 'hisco_code', 'industry', 'sentiment_pattern', 'subjective_pattern']


In [25]:
import nltk
nltk.download([
    "names",
    "stopwords",
    "state_union",
    "twitter_samples",
    "movie_reviews",
    "averaged_perceptron_tagger",
    "vader_lexicon",
    "punkt",
])

from nltk.sentiment import SentimentIntensityAnalyzer

df_comments = df[["text_translation"]].copy()
df_comments.dropna(inplace=True)

sia = SentimentIntensityAnalyzer()

def senti_score(text):
    return sia.polarity_scores(text)["compound"]

df_comments["sentiment_scores"] = df_comments["text_translation"].apply(senti_score)

def label(score):
    if score > 0.05:
        return "positive"
    elif score < -0.05:
        return "negative"
    else:
        return "neutral"

df_comments["sentiment_label"] = df_comments["sentiment_scores"].apply(label)

df_comments.to_csv("sentimentAnalysis.csv", index=False)
print("Sentiment results saved as sentimentAnalysis.csv")


[nltk_data] Downloading package names to /root/nltk_data...
[nltk_data]   Package names is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package state_union to /root/nltk_data...
[nltk_data]   Package state_union is already up-to-date!
[nltk_data] Downloading package twitter_samples to /root/nltk_data...
[nltk_data]   Package twitter_samples is already up-to-date!
[nltk_data] Downloading package movie_reviews to /root/nltk_data...
[nltk_data]   Package movie_reviews is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   

Sentiment results saved as sentimentAnalysis.csv


In [27]:
from google.colab import files
files.download("/content/SMA_Dutch_Social_heatmap/SMA_Dutch_Social_heatmap/sentimentAnalysis.csv")
files.download("/content/SMA_Dutch_Social_heatmap/SMA_Dutch_Social_heatmap/geo_heatmap_full.html")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>